The objective of this code is to extract aspect,aspect category and their sentiment score using pretrained "FINBERT" model which is just BERT model
trained on Financal Dataset

NOTE- It is ideal to run this on KAGGLE as it requires GPU . THE LINK TO THE CODE IS   https://www.kaggle.com/code/adi253/aspect-extraction.

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fiqadataset/tatasentiment.csv
/kaggle/input/fiqadataset/apollosentiment.csv
/kaggle/input/fiqadataset/reliancesentiment.csv


In [2]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification  # import hugguging face libraries for running the model
import torch.nn.functional as F
from transformers import pipeline # to create a pipeline for sentiment score 

Financial aspect words and their categories

In [3]:
## Possible aspect words and their categories 

implicit_aspect_categories = {
    'stock_market': ['surge', 'plummet', 'soar', 'dropped', 'dipped', 'rallied', 'slumped', 
                     'volatile', 'bullish', 'bearish', 'all-time high', 'record low', 
                     'gains', 'losses', 'market sentiment', 'market reaction', 
                     'trading volume', 'price action', 'stake', 'ownership', 'shareholding', 
                     'equity', 'market capitalization', 'trending'],

    'financial_performance': ['beat expectations', 'missed projections', 'cost-cutting', 
                              'revenue growth', 'margin pressure', 'profit warning', 
                              'earnings surprise', 'outperformed', 'underperformed', 
                              'strong performance', 'weak performance', 'increased demand', 
                              'declining demand', 'operating loss', 'profit growth',
                              'financial results', 'net income', 'earnings per share', 
                              'profit margin', 'return on equity', 'gross profit',"expenses","quarter"],

    'investor_sentiment': ['positive outlook', 'negative outlook', 'investor confidence', 
                           'market confidence', 'bearish sentiment', 'bullish sentiment', 
                           'analyst upgrade', 'analyst downgrade', 'investor fears', 
                           'market concerns', 'sentiment shift', 'enthusiasm', 
                           'cautious', 'bull run', 'bear market','shareholders',"shareholders"],

    'market_trends': ['sector rotation', 'market correction', 'upward momentum', 
                      'downward pressure', 'overbought', 'oversold', 'price momentum', 
                      'breakout', 'pullback', 'technical indicators', 'trend reversal', 
                      'market dynamics', 'market breadth', 'momentum', 'futures trend'],

    'economic_indicators': ['inflation concerns', 'interest rate hike', 'monetary policy shift', 
                            'economic slowdown', 'recession fears', 'growth prospects', 
                            'GDP growth', 'inflationary pressure', 'stimulus impact', 
                            'fiscal policy change', 'employment data', 'consumer spending', 
                            'economic forecasts', 'interest rates', 'consumer price index', 
                            'purchasing power'],

    'corporate_news': ['executive change', 'leadership transition', 'strategic shift', 
                       'merger talks', 'acquisition rumors', 'partnership announcement', 
                       'divestiture', 'restructuring', 'new product launch', 
                       'regulatory approval', 'litigation risk', 'legal challenges', 
                       'board decision', 'management change', 'company strategy', 
                       'business restructuring'],

    'financial_crisis': ['liquidity crisis', 'credit freeze', 'market panic', 
                         'volatility spike', 'crisis unfolding', 'financial uncertainty', 
                         'market instability', 'sell-off', 'bank failure', 
                         'economic meltdown', 'financial contagion', 'recession', 'defaults'],

    'commodities': ['oil price surge', 'commodity price hike', 'commodity price drop', 
                    'supply chain disruption', 'global demand', 'commodity inflation', 
                    'raw material cost', 'gold prices', 'silver prices', 
                    'agricultural commodities', 'metals market'],

    'debt_credit': ['debt levels', 'loan approvals', 'bond yields', 'credit risk', 
                    'default rates', 'interest payments', 'refinancing', 
                    'credit downgrade', 'debt restructuring'],

    'regulatory_legal': ['regulatory compliance', 'legal challenges', 'government regulations', 
                         'lawsuit settlement', 'investigation results', 'regulatory scrutiny'],

    'corporate_actions': ['dividend announcement', 'stock split', 'rights issue', 
                          'buyback', 'initial public offering', 'public offering', 
                          'investment strategy'],
    "companies":['Apollo Hospitals',"Reliance","Tata","Tata Motors","Reliance Jio"]

}


In [4]:
# additional sets

new_aspects = {
    'stock_market': ['sensex', 'index', 'points', 'stocks', 'shares', 'nifty'],
    'financial_performance': [ 'turnover', 'earnings report', 'dividend', 'margin', 'profit margin'],
    'corporate_news': ['reliance jio', 'reliance retail', 'ipo', 'fpo', 'capital raise', 'new initiative'],
    'market_trends': [ 'growth stocks', 'blue-chip', 'small-cap', 'large-cap', 'mid-cap'],
    'investor_sentiment': ['mukesh ambani', 'market speculation', 'rumors', 'investor reactions'],
    'economic_indicators': ['gdp numbers', 'inflation rate', 'monetary easing', 'economic recovery','net profit']
 
}


for category, new_terms in new_aspects.items():
    implicit_aspect_categories[category].extend(new_terms)

Load  spacy model and FINBERT model for aspect extraction

In [5]:
import spacy  # Import the spaCy library for natural language processing
from transformers import pipeline  # Import the pipeline function from Hugging Face Transformers

# Load the English language model from spaCy
nlp = spacy.load('en_core_web_sm')  # 'en_core_web_sm' is a small English model for various NLP tasks

# Initialize a sentiment analysis pipeline using the FinBERT model from ProsusAI
# The model is set to use a GPU ('cuda') if available; otherwise, it will use CPU by default
sentiment_analyzer = pipeline("sentiment-analysis", model="ProsusAI/finbert", device="cuda")


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
'''def absa(text):
    text = text

    doc = nlp(text)

    def find_implicit_aspects(sentence, categories):
        aspects_found = []
    

        sentence_text = sentence.text.lower()
    

        for category, keywords in categories.items():
            for keyword in keywords:
                if keyword.lower() in sentence_text:
                    aspects_found.append((keyword, category))
        return aspects_found


    implicit_aspects = []
    for sent in doc.sents:
        implicit_aspects.extend(find_implicit_aspects(sent, implicit_aspect_categories))


    aspect_sentiment = []
    for aspect, category in implicit_aspects:
        for sent in doc.sents:
            if aspect.lower() in sent.text.lower():  
                sentiment = sentiment_analyzer(sent.text)[0]
                aspect_sentiment.append((aspect, category, sentiment['label'], sentiment['score']))

    if aspect_sentiment:
        #for aspect, category, sentiment, score in aspect_sentiment:
            #print(f"Aspect: {aspect} (Category: {category}), Sentiment: {sentiment}, Score: {score:.2f}")
        return aspect_sentiment
    #else:
        #print("No implicit aspects found in the text.") '''

'def absa(text):\n    text = text\n\n    doc = nlp(text)\n\n    def find_implicit_aspects(sentence, categories):\n        aspects_found = []\n    \n\n        sentence_text = sentence.text.lower()\n    \n\n        for category, keywords in categories.items():\n            for keyword in keywords:\n                if keyword.lower() in sentence_text:\n                    aspects_found.append((keyword, category))\n        return aspects_found\n\n\n    implicit_aspects = []\n    for sent in doc.sents:\n        implicit_aspects.extend(find_implicit_aspects(sent, implicit_aspect_categories))\n\n\n    aspect_sentiment = []\n    for aspect, category in implicit_aspects:\n        for sent in doc.sents:\n            if aspect.lower() in sent.text.lower():  \n                sentiment = sentiment_analyzer(sent.text)[0]\n                aspect_sentiment.append((aspect, category, sentiment[\'label\'], sentiment[\'score\']))\n\n    if aspect_sentiment:\n        #for aspect, category, sentiment, scor

This function performs Aspect-Based Sentiment Analysis (ABSA) on a given text to identify financial-related aspects and find its category and also find  sentiment score  for each aspect within a defined context window



In [8]:
def absa(text, window_size=8):
    # Process the input text using the spaCy NLP model
    doc = nlp(text)
    
    # Function to find implicit aspects in a given sentence based on predefined categories and keywords
    def find_implicit_aspects(sentence, categories):
        aspects_found = []  # List to store found aspects
        sentence_text = sentence.text.lower()  # Convert the sentence text to lowercase for comparison
        for category, keywords in categories.items():  # Iterate through each category and its keywords
            for keyword in keywords:
                # Check if the keyword is present in the sentence
                if keyword.lower() in sentence_text:
                    aspects_found.append((keyword, category))  # Append the found aspect and its category
        return aspects_found  # Return the list of found aspects
    
    # Function to get a context window around a specific aspect in a sentence
    def get_windowed_context(aspect, sentence, window_size=5):
        # Tokenize the sentence using spaCy
        tokens = [token.text for token in sentence]
        aspect_tokens = nlp(aspect)  # Tokenize the aspect
        aspect_length = len(aspect_tokens)  # Get the length of the aspect in tokens
        
        # Iterate over the tokens to find the aspect
        for i in range(len(tokens) - aspect_length + 1):
            if ' '.join(tokens[i:i+aspect_length]).lower() == aspect.lower():  # Check for aspect match
                # Calculate the start and end indices for the context window
                start = max(0, i - window_size)  # Ensure start index is not negative
                end = min(len(tokens), i + aspect_length + window_size)  # Ensure end index is within bounds
                return ' '.join(tokens[start:end])  # Return the windowed context as a string
        return None  # Return None if aspect is not found
    
    aspect_sentiment = []  # List to store results of aspect sentiment analysis
    for sent in doc.sents:  # Iterate through each sentence in the document
        # Find implicit aspects in the current sentence
        implicit_aspects = find_implicit_aspects(sent, implicit_aspect_categories)
        for aspect, category in implicit_aspects:  # Iterate through found aspects
            windowed_context = get_windowed_context(aspect, sent, window_size)  # Get the context window for the aspect
            if windowed_context:  # If a context window was found
                # Analyze sentiment of the windowed context using the sentiment analysis pipeline
                sentiment = sentiment_analyzer(windowed_context)[0]  # Get the sentiment analysis result
                # Append the aspect, category, sentiment label, and sentiment score to the results list
                aspect_sentiment.append((aspect, category, sentiment['label'], sentiment['score']))
                    
    return aspect_sentiment  # Return the list of aspect sentiments


In [9]:
df=pd.read_csv("/kaggle/input/fiqadataset/tatasentiment.csv")

In [10]:
df["news"][200].split(" | ")

['PNB  is expected to improve upon its performance this quarter as its balance sheet improves due to inflow of funds due to Bhushan Steel’s acquisition by Tata Steel',
 'Tata Motors will kick off the launch of new products with SUV Harrier in the fourth quarter of the current fiscal']

In [11]:
absa(df["news"][3])

[('Tata', 'companies', 'neutral', 0.8824670910835266),
 ('Tata Motors', 'companies', 'neutral', 0.8078752160072327)]

In [12]:
df["aspect_labelling"] = None  
df = df.astype({"aspect_labelling": 'object'})  

In [ ]:
#sentences=df["news"][4].split(" | ")

In [ ]:
#set(sentences)

In [14]:
for i in range(len(df)):
    sentences=list(set(df.loc[i,"news"].split(" | ")))
    aspects=[]
    for news in sentences:
        result=absa(news)
        aspects.append(result)
    df.at[i,"aspect_labelling"]=aspects

In [15]:
df["aspect_labelling"][0]

[[('sensex', 'stock_market', 'negative', 0.9329291582107544),
  ('Tata', 'companies', 'neutral', 0.9313399791717529),
  ('Tata Motors', 'companies', 'positive', 0.5572634935379028)],
 [('stocks', 'stock_market', 'neutral', 0.9116292595863342),
  ('Tata', 'companies', 'positive', 0.9471097588539124)],
 [('Tata', 'companies', 'neutral', 0.9239994883537292),
  ('Tata Motors', 'companies', 'neutral', 0.9239994883537292)]]

In [ ]:
df["news"][0]

In [16]:
df.head()

,Unnamed: 0,date,title,feed_type,link,news,Predicted Sentiment,Sentiment Probability,aspect_labelling
0,0,2016-10-05,Steel stocks rally as govt extends MIP on 66 s...,news-article | news-article | news-article,https://www.moneycontrol.com/news/business/sto...,"Stocks like Hindustan Zinc, Hindalco, Jindal S...",positive,0.752132,"[[(sensex, stock_market, negative, 0.932929158..."
1,1,2017-12-11,BSE Midcap index hits new high | Thyssenkrupp ...,news-article | news-article | news-article,http://www.business-standard.com/article/marke...,"Wockhdart, M&M Financial Services, Tata Commun...",positive,0.941678,"[[(Tata, companies, neutral, 0.777573883533477..."
2,2,2017-12-14,Tata Motors delivers first batch of Tigor EVs ...,news-article,https://economictimes.indiatimes.com/industry/...,"Guenter Butschek, CEO & MD, Tata Motors handed...",neutral,0.869739,"[[(Tata, companies, neutral, 0.900660037994384..."
3,3,2017-12-15,Tata Motors is driving Ace on the millennial t...,news-article,http://www.business-standard.com/article/compa...,How Tata Motors is leveraging its 12-year expe...,neutral,0.535472,"[[(Tata, companies, neutral, 0.882467091083526..."
4,4,2017-12-16,‘System error’ hits BEST’s 750 MW power tender,news-article,http://www.thehindubusinessline.com/news/syste...,Tata Power likely to benefit,neutral,0.652322,"[[(Tata, companies, neutral, 0.65232253074646)]]"


In [ ]:
df.columns

In [ ]:
df.iloc[12]["aspect_labelling"]

In [ ]:
df.drop(['Unnamed: 0','title','feed_type','link'],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df.to_csv("/kaggle/working/tata_aspect.csv",index=False)

In [ ]:
result[2]

In [ ]:
absa(df["news"][0])

In [ ]:
'''from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline


model_name = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

for aspect in ['stake']:
       print(aspect, classifier('So far, Reliance Industries has Hotstar and Roy Kapur Films on board for content on Reliance Jio, and bought 24.9% stake in Alt Balaji for Rs413 crore',  text_pair=aspect))'''
